In [1]:
#ENV["PYTHON"] = "/opt/homebrew/Caskroom/miniforge/base/envs/sar_ais_2/bin/python"
#using Pkg
#Pkg.build("PyCall")


In [2]:
using Pkg
using DotEnv
using PyCall
include("src/test_name.jl")
import .Test_name
DotEnv.config();

┌ Warning: Replacing docs for `Main.Test_name.Download.Metadata.to_rcm_time :: Tuple{String}` in module `Main.Test_name.Download.Metadata`
└ @ Base.Docs docs/Docs.jl:240
┌ Warning: Replacing docs for `Main.Test_name.Download.Metadata.bounding_box_to_geojson :: Tuple{}` in module `Main.Test_name.Download.Metadata`
└ @ Base.Docs docs/Docs.jl:240
┌ Warning: Replacing docs for `Main.Test_name.Download.Metadata.coordinate_to_geojson :: Tuple{Float64, Float64}` in module `Main.Test_name.Download.Metadata`
└ @ Base.Docs docs/Docs.jl:240
┌ Warning: Replacing docs for `Main.Test_name.Download.Metadata.wkt_to_tuples :: Tuple{Any}` in module `Main.Test_name.Download.Metadata`
└ @ Base.Docs docs/Docs.jl:240


In [3]:
# Define the base URL for the Sentinel-1 data collection
start_time = Test_name.Download.Metadata.to_rcm_time("2022-10-10");
end_time = Test_name.Download.Metadata.to_rcm_time("2022-10-11");
box = "70,75,5,10"
split_strings = split(box, ",")
lat_min, lat_max, lon_min, lon_max  = parse.(Int, split_strings)
#lat_min, lat_max, lon_min, lon_max = map(float, bbox.split(','))
area = Test_name.Download.Metadata.bounding_box_to_geojson(lat_min=lat_min, lat_max=lat_max, lon_min=lon_min, lon_max=lon_max);



In [4]:

 area = Test_name.Download.Metadata.coordinate_to_geojson(lat_min,lon_min)

UndefVarError: UndefVarError: bbox not defined

In [ ]:
res = download_metadata_rcm(username="Aalling93", password = "Security1234!");

In [148]:




@pyimport eodms_rapi
function download_metadata_rcm(; area, 
    start_datetime::String="2022-10-10", 
    end_datetime::String="2022-10-11", 
    max_records::Int64=10,
    collection::String = "RCM",
    polarization::Array{String} = ["HH", "HH HV", "HH HV VH VV", "HH VV",],
    product_type::Array{String} =  ["GCC", "GCD", "GRC", "GRD"],
    bean_mode::String = "ScanSAR 50%",
    spatial_resolution::String = "50",
    username::String = "username",
    password::String = "password",
    verbose::Int = 1)  # Adjust product type if needed
    
    # Define the base URL for the Sentinel-1 data collection
    start_time = Test_name.Download.Metadata.to_rcm_time(start_datetime);
    end_time = Test_name.Download.Metadata.to_rcm_time(end_datetime);
    


    #filters = Dict(
    #    "Product Type" => ("=", product_type),
    #    "Polarization" => ("=", polarization),
    #    "Beam Mode Type" => ("=", bean_mode)  # Assuming this is what you meant instead of repeating "Beam Mode Type"
    #)
    filters = Dict(
        "Product Type" => ("like", product_type),
        "Spatial Resolution" => ("=", spatial_resolution),
        "Polarization" => ("like", polarization),
        "Beam Mode" => ("contains", bean_mode)  # Assuming this is what you meant instead of repeating "Beam Mode Type"
    )
    #filters = Dict(
    # #   "Product Type" => ("contains", "GRD"),
    #    "Spatial Resolution" => ("=", "50"),
    #    "Polarization" => ("contains", "HH"),
    #    "Beam Mode" => ("contains", "ScanSAR 50%")  # Assuming this is what you meant instead of repeating "Beam Mode Type"
    #)


    
    dates = [Dict("start" => start_time, "end" => end_time)]

    # Import the Python module
    #@pyimport pandas as pd
    
    #features = area, f
    # Create the EODMSRAPI object using Python code
    rapi = eodms_rapi.EODMSRAPI(username, password)
    # Call the search method from the Python module #filters, feats, dates, 
    rapi.search(collection = "RCMImageProducts", filters = filters, features = [],dates = dates, max_results=max_records)
    # Get the results from the search
    res = rapi.get_results("full")

    return res
end



download_metadata_rcm (generic function with 1 method)

In [149]:
res = download_metadata_rcm(username="Aalling93", password = "Security1234!");

UndefKeywordError: UndefKeywordError: keyword argument area not assigned

In [150]:
res[1]

Dict{Any, Any} with 99 entries:
  "satellite"                   => "RCM"
  "productDatum"                => ""
  "rapiOrderUrl"                => "https://www.eodms-sgdot.nrcan-rncan.gc.ca/w…
  "serviceUuid"                 => "62f0e816-8006-4768-8f32-6ef4008e6895"
  "openData"                    => "false"
  "sampledPixelSpacing"         => "20"
  "subcollectionId"             => "2"
  "providerId"                  => "1"
  "orbitDataSource"             => "Downlinked"
  "thumbnailUrl"                => "https://www.eodms-sgdot.nrcan-rncan.gc.ca/w…
  "collectionId"                => "RCMImageProducts"
  "specialHandlingInstructions" => "No special handling required, use per EULA"
  "datasetId"                   => "2291684_1"
  "eulaFilename"                => "license.pdf"
  "processorVersion"            => "RCM PGS 7.06"
  "polarization"                => "HH HV"
  "numberOfRangeLooks"          => "4"
  "metadataFullUrl"             => "https://www.eodms-sgdot.nrcan-rncan.gc.ca/w…
 

In [136]:
# Function to create a directory and its parents if they don't exist
function make_directory_recursive(path::String)
    # Split the path into parts
    path_parts = split(path, "/")
    partial_path = ""

    # Iterate through parts of the path
    for part in path_parts
        # Update the partial path
        partial_path = joinpath(partial_path, part)

        # Check if this part of the path exists
        if !isdir(partial_path) && partial_path != ""
            # Create the directory if it doesn't exist
            mkdir(partial_path)
        end
    end
end

make_directory_recursive (generic function with 1 method)

In [145]:
@pyimport eodms_rapi
function download_product_rcm(; username::String, password::String, product_result::Dict{Any, Any}, destination::String) 

    #make_directory_recursive(destination)
    rapi = eodms_rapi.EODMSRAPI(username, password)

    order_res = rapi.order(product_result)
    download = rapi.download(order_res, destination)
    rapi.clear_results()
    return nothing 
end



download_product_rcm (generic function with 1 method)

In [146]:
path = "/Users/kaaso/Documents/coding/JuliaEO2024/sss"
path = "/Users/kaaso/Documents/coding/JuliaEO2024/test_rcm_data"

"/Users/kaaso/Documents/coding/JuliaEO2024/test_rcm_data"

In [139]:
make_directory_recursive(path)

In [147]:
download_product_rcm(username="Aalling93", password = "Security1234!",product_result = res[1],destination = path)

PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= /Users/kaaso/.julia/packages/PyCall/KLzIO/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'TypeError'>
TypeError('string indices must be integers')
  File "/opt/homebrew/Caskroom/miniforge/base/envs/sar_ais_2/lib/python3.9/site-packages/eodms_rapi/eodms.py", line 3099, in order
    item = {'collectionId': r[coll_key],


In [124]:
res[1]

Dict{Any, Any} with 99 entries:
  "satellite"                   => "RCM"
  "productDatum"                => ""
  "rapiOrderUrl"                => "https://www.eodms-sgdot.nrcan-rncan.gc.ca/w…
  "serviceUuid"                 => "62f0e816-8006-4768-8f32-6ef4008e6895"
  "openData"                    => "false"
  "sampledPixelSpacing"         => "20"
  "subcollectionId"             => "2"
  "providerId"                  => "1"
  "orbitDataSource"             => "Downlinked"
  "thumbnailUrl"                => "https://www.eodms-sgdot.nrcan-rncan.gc.ca/w…
  "collectionId"                => "RCMImageProducts"
  "specialHandlingInstructions" => "No special handling required, use per EULA"
  "datasetId"                   => "2291684_1"
  "eulaFilename"                => "license.pdf"
  "processorVersion"            => "RCM PGS 7.06"
  "polarization"                => "HH HV"
  "numberOfRangeLooks"          => "4"
  "metadataFullUrl"             => "https://www.eodms-sgdot.nrcan-rncan.gc.ca/w…
 

make_directory_recursive (generic function with 1 method)

In [126]:
make_directory_recursive(path)

In [ ]:
# Download images to a specific destination
dest = "C:\\TEMP"
dn_res = rapi.download(order_res, dest)

# Print results
rapi.print_results(dn_res)

# Clear search results
rapi.clear_results()

In [108]:
df = Test_name.Download.Metadata.vector_of_dicts_to_dataframe(res);

In [110]:
df

Row,satellite,productDatum,rapiOrderUrl,serviceUuid,openData,sampledPixelSpacing,subcollectionId,providerId,orbitDataSource,thumbnailUrl,collectionId,specialHandlingInstructions,datasetId,eulaFilename,processorVersion,polarization,numberOfRangeLooks,metadataFullUrl,sensorId,sensorFolder,quicklookFullName,clientOrderNumber,acquisitionEndDate,metadataFullName,beamModeDefinitionId,sipSize,geometry_crs_properties_name,creationDate,spatialResolution,downlinkSegmentId,specialHandlingRequired,overviewUrl,sampleType,dataType,geometry_coordinates_10,incidenceAngle,inter-polarizationMatricesCorrection,supplierOrderItemNumber,inputDatasetFacility,geometry_coordinates_4,lutApplied,dopplerAnomalyGrid,lookOrientation,archiveVisibilityStartDate,thisRecordUrl,productMapZone,beamMnemonic,relativeOrbit,beamModeVersion,securityClassification,collectionTitle,processingFacility,isGeorectified,calibrationIssues,geometry_coordinates_7,geometry_type,metadata,buyerId,polarizationInProduct,acquisitionStartDate,notifiedClientEmailList,absoluteOrbit,recordId,geometry_coordinates_1,productFormat,geometry_coordinates_2,orderExecuteUrl,geometry_coordinates_9,productMapProjection,beamModeDescription,geographicalArea,vapEnabled,processingPriority,featureId,relatedProducts,geometry_coordinates_5,productOrientation,geometry_coordinates_8,orbitDataFilename,isOrderable,processingLevel,supplierOrderNumber,satelliteId,options,geometry_crs_type,publicGood,numberOfAzimuthLooks,productEllipsoid,sequenceId,geometry_coordinates_3,title,geodeticTerrainHeight,productId,geometry_coordinates_6,ownerId,declassificationDate,processingMode,processingDate,wktGeometry,withinOrbitTube,⋯
,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,String,Float64,String,String,String,String,Float64,String,String,String,String,String,String,String,String,String,String,String,String,Bool,String,Float64,String,String,String,String,String,String,String,String,Float64,String,Float64,String,Float64,String,String,String,String,String,String,String,Float64,String,Float64,String,Bool,String,String,String,String,String,String,String,String,String,Float64,String,String,String,Float64,String,String,String,String,String,String,⋯
1,RCM,,https://www.eodms-sgdot.nrcan-rncan.gc.ca/wes/rapi/order/direct?collection=RCMImageProducts&recordId=14068734&destination=fill_me_in,62f0e816-8006-4768-8f32-6ef4008e6895,false,20,2,1,Downlinked,https://www.eodms-sgdot.nrcan-rncan.gc.ca/wes/getObject?FeatureID=62f0e816-8006-4768-8f32-6ef4008e6895-14068734&ObjectType=Thumbview&collectionId=RCMImageProducts&AFB=true,RCMImageProducts,"No special handling required, use per EULA",2291684_1,license.pdf,RCM PGS 7.06,HH HV,4,https://www.eodms-sgdot.nrcan-rncan.gc.ca/wes/getArchivedMetadata?FeatureID=62f0e816-8006-4768-8f32-6ef4008e6895-14068734&MetadataFileName=/rcm/sar/l1/2022/noamer/med50/rcm_20221010_N4992W13238/ec9bbb44-f900-5acc-9f20-05c75c072c76&collectionId=RCMImageProducts&fileName=catalog/image/rcm/sar/l1/2022/noamer/med50/rcm_20221010_N4992W13238/ec9bbb44-f900-5acc-9f20-05c75c072c76/https://www.eodms-sgdot.nrcan-rncan.gc.ca/wes/getArchivedMetadata?FeatureID=62f0e816-8006-4768-8f32-6ef4008e6895-14068734&MetadataFileName=/rcm/sar/l1/2022/noamer/med50/rcm_20221010_N4992W13238/ec9bbb44-f900-5acc-9f20-05c75c072c76&collectionId=RCMImageProducts&fileName=catalog/image/rcm/sar/l1/2022/noamer/med50/rcm_20221010_N4992W13238/ec9bbb44-f900-5acc-9f20-05c75c072c76/product.xml,88,med50,/rcm/sar/l1/2022/noamer/med50/rcm_20221010_N4992W13238/355239/ec9bbb44-f900-5acc-9f20-05c75c072c76.png,,2022-10-10T02:39:02 +0000,/rcm/sar/l1/2022/noamer/med50/rcm_20221010_N4992W13238/ec9bbb44-f900-5acc-9f20-05c75c072c76,22,867,EPSG:4326,2022-10-10T02:47:58 +0000,50,5094546_5099001,false,http://was-eodms.compusult.net/wes/images/No_Data_Available.png,Magnitude Detected,Integer